In [857]:
import numpy as np

from enum import Enum
from typing import List


In [858]:
class Signs(Enum):
    LESS = 1
    EQUAL = 2
    GREAT = 3


class TaskType(Enum):
    MIN = 1
    MAX = 2


class Inequality:
    def __init__(self, coeff: List[int], sign: Signs, free: int):
        self.coeff = coeff
        self.sign = sign
        self.free = free


class Task:
    def __init__(self, main_coeff: List[int], task_type: TaskType, inequals: List[Inequality]):
        self.main_coeff = main_coeff
        self.task_type = task_type
        self.inequals = inequals


In [859]:
def check(actual, expected):
    if isinstance(actual, list):
        ok = True
        for i in range(len(actual)):
            if actual[i] != expected[i]:
                ok = False
        if not ok:
            print('points are different')
    else:
        if actual != expected:
            print(f'not equal. expected: {expected}, got: {actual}')
        # raise RuntimeError(f'not equal. expected: {expected}, got: {actual}')


In [860]:
z = np.array(
    [
        [1, 4, 9, 3],
        [2, 7, 8, 6],
        [5, 3, 0, 1],
    ]
)

print(z)

z = np.c_[z, [1, 2, 3]]

print(z)


[[1 4 9 3]
 [2 7 8 6]
 [5 3 0 1]]
[[1 4 9 3 1]
 [2 7 8 6 2]
 [5 3 0 1 3]]


In [861]:
print(np.array([0] * 3 + [-1] * 2))
print(np.append([1, 2], 3))

[ 0  0  0 -1 -1]
[1 2 3]


In [862]:
def simplex1(matrix_a, column_b, functional, initial_basis):
    data = np.c_[matrix_a, column_b * -1]

    n = matrix_a.shape[1]
    m = matrix_a.shape[0]

    print(data)
    print(functional)
    print(data.shape)
    # TODO rewrite gauss

    # TODO преобразование initial_basis

    # to ones on basis
    m_to_basis = [-1] * m
    cnt = 0
    for ind in range(n):
        if initial_basis[ind] == 1:
            if data[cnt][ind] == 0:
                for i in range(cnt + 1, m):
                    if data[i][ind] != 0:
                        data[cnt] += data[i]
            data[cnt] /= data[cnt][ind]

            for i in range(m):
                if i == cnt:
                    continue
                data[i] -= data[cnt] * data[i][ind]
            m_to_basis[cnt] = ind
            cnt += 1

    # recalc functional
    cnt = 0
    for ind in range(n):
        if initial_basis[ind] == 1:
            functional -= data[cnt] * (functional[ind] / data[cnt][ind])
            cnt += 1

    print("after gauss")
    print(data)
    print(functional)
    print("----------")

    for cyc in range(100):
        ind_in_basis = -1
        for i in range(n):
            if functional[i] > 0:
                ind_in_basis = i
                break
        if ind_in_basis == -1:
            print('complete')
            break
        ind_out_basis = -1
        prev_rez = -1e10
        for i in range(m):
            if data[i][ind_in_basis] == 0:
                continue
            tt1 = data[i][-1] / data[i][ind_in_basis]
            if 0 >= tt1 > prev_rez:
                ind_out_basis = i
                prev_rez = tt1
        if ind_out_basis == -1:
            print('unlimited')
            break
        # in_basis[ind_out_basis] = 0
        # in_basis[ind_in_basis] = 1
        m_to_basis[ind_out_basis] = ind_in_basis

        data[ind_out_basis] /= data[ind_out_basis][ind_in_basis]
        for i in range(m):
            if i != ind_out_basis:
                data[i] -= data[ind_out_basis] * data[i][ind_in_basis]

        functional -= data[ind_out_basis] * functional[ind_in_basis]

        print(cyc)
        print(ind_in_basis, ind_out_basis)
        print(data)
        print(functional)
        print("----------")

    pnt = [0] * n
    for i in range(m):
        pnt[m_to_basis[i]] = -data[i][-1]

    for i in range(n):
        if abs(pnt[i]) < 1e-10:
            pnt[i] = 0

    return functional[-1], pnt

In [863]:
def solve(task: Task, initial_solution: List[int] = None, initial_basis: List[int] = None):
    # n = len(task.main_coeff) + sum()
    m = len(task.inequals)
    il = len(task.inequals[0].coeff)

    # transform data from dsl to np.array
    matrix_a = np.array(
        [inequality.coeff for inequality in task.inequals],
        dtype=np.float64
    )
    column_b = np.array(
        [inequality.free for inequality in task.inequals],
        dtype=np.float64
    )

    for i in range(m):
        inequality = task.inequals[i]
        if inequality.sign == Signs.GREAT:
            matrix_a[i] *= -1
            column_b[i] *= -1
            # TODO check: sign changed from > to <
        if inequality.sign != Signs.EQUAL:
            clmn = np.zeros(m)
            clmn[i] = 1
            matrix_a = np.c_[
                matrix_a,
                clmn
            ]
            task.main_coeff.append(0)

    main_functional = np.array(task.main_coeff, dtype=np.float64)
    if task.task_type == TaskType.MIN:
        main_functional *= -1

    if initial_basis is None:
        matrix_y = np.eye(m)
        for i in range(m):
            if column_b[i] < 0:
                matrix_y[i][i] = -1
        m_AY = np.c_[matrix_a, matrix_y]
        a_functional = [0] * main_functional.shape[0] + [-1] * m + [0]
        a_in_func = [0] * main_functional.shape[0] + [1] * m
        opt_value, point = simplex1(m_AY, column_b, a_functional, a_in_func)

        print(">", opt_value, point)
        if abs(opt_value) > 1e-10:
            print("not feasible")
            return 0

        initial_basis = [1 if x != 0 else 0 for x in point[:-m]]
        # remove extra zero
        rrr = m - sum(initial_basis)
        ind = 0
        while rrr > 0:
            if initial_basis[ind] == 0:
                initial_basis[ind] = 1
                rrr -= 1
            ind += 1

        print(initial_basis)

    main_functional = np.append(main_functional, 0)

    opt_value, pnt = simplex1(matrix_a, column_b, main_functional, initial_basis)

    pnt = pnt[:il]

    result_mn = 1 if task.task_type == TaskType.MAX else -1
    print("functional: ", result_mn * opt_value)
    print("in point:", pnt)

    return result_mn * opt_value, pnt


In [864]:
task_1 = Task(
    [-6, -1, -4, 5], TaskType.MIN,
    [
        Inequality([3, 1, -1, 1], Signs.EQUAL, 4),
        Inequality([5, 1, 1, -1], Signs.EQUAL, 4)
    ]
)

result_1_value, result_1_point = solve(
    task_1,
    [1, 0, 0, 1],
    [1, 0, 0, 1]
)

check(result_1_value, -4)
check(result_1_point, [0, 4, 0, 0])

[[ 3.  1. -1.  1. -4.]
 [ 5.  1.  1. -1. -4.]]
[ 6.  1.  4. -5.  0.]
(2, 5)
after gauss
[[ 1.    0.25  0.    0.   -1.  ]
 [-0.    0.25 -1.    1.   -1.  ]]
[ 0.    0.75 -1.    0.    1.  ]
----------
0
1 0
[[ 4.  1.  0.  0. -4.]
 [-1.  0. -1.  1.  0.]]
[-3.  0. -1.  0.  4.]
----------
complete
functional:  -4.0
in point: [0, 4.0, 0, 0]


In [865]:
task_1_2 = Task(
    [-6, -1, -4, 5], TaskType.MIN,
    [
        Inequality([3, 1, -1, 1], Signs.EQUAL, 4),
        Inequality([5, 1, 1, -1], Signs.EQUAL, 4)
    ]
)

result_1_2_value, result_1_2_point = solve(
    task_1_2
)

check(result_1_2_value, -4)
check(result_1_2_point, [0, 4, 0, 0])


[[ 3.  1. -1.  1.  1.  0. -4.]
 [ 5.  1.  1. -1.  0.  1. -4.]]
[0, 0, 0, 0, -1, -1, 0]
(2, 7)
after gauss
[[ 3.  1. -1.  1.  1.  0. -4.]
 [ 5.  1.  1. -1.  0.  1. -4.]]
[ 8.  2.  0.  0.  0.  0. -8.]
----------
0
0 1
[[ 0.   0.4 -1.6  1.6  1.  -0.6 -1.6]
 [ 1.   0.2  0.2 -0.2  0.   0.2 -0.8]]
[ 0.   0.4 -1.6  1.6  0.  -1.6 -1.6]
----------
1
1 0
[[ 0.   1.  -4.   4.   2.5 -1.5 -4. ]
 [ 1.   0.   1.  -1.  -0.5  0.5  0. ]]
[ 0.  0.  0.  0. -1. -1.  0.]
----------
complete
> 0.0 [0, 4.0, 0, 0, 0, 0]
[1, 1, 0, 0]
[[ 3.  1. -1.  1. -4.]
 [ 5.  1.  1. -1. -4.]]
[ 6.  1.  4. -5.  0.]
(2, 5)
after gauss
[[ 1.  0.  1. -1.  0.]
 [-0.  1. -4.  4. -4.]]
[ 0.  0.  2. -3.  4.]
----------
0
2 0
[[ 1.  0.  1. -1.  0.]
 [ 4.  1.  0.  0. -4.]]
[-2.  0.  0. -1.  4.]
----------
complete
functional:  -4.0
in point: [0, 4.0, 0, 0]


In [866]:
#TODO дробные числа

task_2 = Task(
    [-1, -2, -3, 1], TaskType.MIN,
    [
        Inequality([1, -3, -1, -2], Signs.EQUAL, -4),
        Inequality([1, -1, 1, 0], Signs.EQUAL, 0)
    ]
)

result_2_value, result_2_point = solve(
    task_2,
    [0, 1, 1, 0],
    [0, 1, 1, 0]
)

check(result_2_value, -6)
check(result_2_point, [2, 2, 0, 0])


[[ 1. -3. -1. -2.  4.]
 [ 1. -1.  1.  0. -0.]]
[ 1.  2.  3. -1.  0.]
(2, 5)
after gauss
[[-0.5  1.   0.   0.5 -1. ]
 [ 0.5  0.   1.   0.5 -1. ]]
[ 0.5  0.   0.  -3.5  5. ]
----------
0
0 1
[[ 0.  1.  1.  1. -2.]
 [ 1.  0.  2.  1. -2.]]
[ 0.  0. -1. -4.  6.]
----------
complete
functional:  -5.999999999999999
in point: [1.9999999999999996, 1.9999999999999998, 0, 0]
not equal. expected: -6, got: -5.999999999999999
points are different


In [867]:
task_2_max = Task(
    [-1, -2, -3, 1], TaskType.MAX,
    [
        Inequality([1, -3, -1, -2], Signs.EQUAL, -4),
        Inequality([1, -1, 1, 0], Signs.EQUAL, 0)
    ]
)

result_2_max_value, result_2_max_point = solve(
    task_2_max,
    [0, 1, 1, 0],
    [0, 1, 1, 0]
)

check(result_2_max_value, 2)
check(result_2_max_point, [0, 0, 0, 2])


[[ 1. -3. -1. -2.  4.]
 [ 1. -1.  1.  0. -0.]]
[-1. -2. -3.  1.  0.]
(2, 5)
after gauss
[[-0.5  1.   0.   0.5 -1. ]
 [ 0.5  0.   1.   0.5 -1. ]]
[-0.5  0.   0.   3.5 -5. ]
----------
0
3 0
[[-1.  2.  0.  1. -2.]
 [ 1. -1.  1.  0.  0.]]
[ 3. -7.  0.  0.  2.]
----------
1
0 1
[[ 0.  1.  1.  1. -2.]
 [ 1. -1.  1.  0.  0.]]
[ 0. -4. -3.  0.  2.]
----------
complete
functional:  2.0
in point: [0, 0, 0, 2.0]


In [868]:
# TODO fix gauss

task_3 = Task(
    [-1, -2, -1, 3, -1], TaskType.MIN,
    [
        Inequality([1, 1, 0, 2, 1], Signs.EQUAL, 5),
        Inequality([1, 1, 1, 3, 2], Signs.EQUAL, 9),
        Inequality([0, 1, 1, 2, 1], Signs.EQUAL, 6)
    ]
)

result_3_value, result_3_point = solve(
    task_3,
    # [0, 0, 1, 2, 1],
    # [0, 0, 1, 1, 1]
)

check(result_3_value, -11)
check(result_3_point, [3, 2, 4, 0, 0])


[[ 1.  1.  0.  2.  1.  1.  0.  0. -5.]
 [ 1.  1.  1.  3.  2.  0.  1.  0. -9.]
 [ 0.  1.  1.  2.  1.  0.  0.  1. -6.]]
[0, 0, 0, 0, 0, -1, -1, -1, 0]
(3, 9)
after gauss
[[ 1.  1.  0.  2.  1.  1.  0.  0. -5.]
 [ 1.  1.  1.  3.  2.  0.  1.  0. -9.]
 [ 0.  1.  1.  2.  1.  0.  0.  1. -6.]]
[  2.   3.   2.   7.   4.   0.   0.   0. -20.]
----------
0
0 0
[[ 1.  1.  0.  2.  1.  1.  0.  0. -5.]
 [ 0.  0.  1.  1.  1. -1.  1.  0. -4.]
 [ 0.  1.  1.  2.  1.  0.  0.  1. -6.]]
[  0.   1.   2.   3.   2.  -2.   0.   0. -10.]
----------
1
1 0
[[ 1.  1.  0.  2.  1.  1.  0.  0. -5.]
 [ 0.  0.  1.  1.  1. -1.  1.  0. -4.]
 [-1.  0.  1.  0.  0. -1.  0.  1. -1.]]
[-1.  0.  2.  1.  1. -3.  0.  0. -5.]
----------
2
2 2
[[ 1.  1.  0.  2.  1.  1.  0.  0. -5.]
 [ 1.  0.  0.  1.  1.  0.  1. -1. -3.]
 [-1.  0.  1.  0.  0. -1.  0.  1. -1.]]
[ 1.  0.  0.  1.  1. -1.  0. -2. -3.]
----------
3
0 1
[[ 0.  1.  0.  1.  0.  1. -1.  1. -2.]
 [ 1.  0.  0.  1.  1.  0.  1. -1. -3.]
 [ 0.  0.  1.  1.  1. -1.  1.  0. -4.]]
[ 0.

In [869]:
task4 = Task(
    [-1, -1, -1, 1, -1], TaskType.MIN,
    [
        Inequality([1, 1, 2, 0, 0], Signs.EQUAL, 4),
        Inequality([0, -2, -2, 1, -1], Signs.EQUAL, -6),
        Inequality([1, -1, 6, 1, 1], Signs.EQUAL, 12)
    ]
)

result_4_value, result_4_point = solve(
    task4,
    [1, 1, 2, 0, 0],
    [1, 1, 1, 0, 0],
)

check(result_4_value, -10)
check(result_4_point, [4, 0, 0, 6, 12])


[[  1.   1.   2.   0.   0.  -4.]
 [  0.  -2.  -2.   1.  -1.   6.]
 [  1.  -1.   6.   1.   1. -12.]]
[ 1.  1.  1. -1.  1.  0.]
(3, 6)
after gauss
[[ 1.          0.          0.          0.5        -0.83333333  1.33333333]
 [-0.          1.          0.         -0.5         0.16666667 -0.66666667]
 [ 0.          0.          1.          0.          0.33333333 -2.33333333]]
[ 0.          0.          0.         -1.          1.33333333  1.66666667]
----------
0
4 0
[[-1.2 -0.  -0.  -0.6  1.  -1.6]
 [ 0.2  1.   0.  -0.4  0.  -0.4]
 [ 0.4  0.   1.   0.2  0.  -1.8]]
[ 1.6  0.   0.  -0.2  0.   3.8]
----------
1
0 1
[[ 0.  6.  0. -3.  1. -4.]
 [ 1.  5.  0. -2.  0. -2.]
 [ 0. -2.  1.  1.  0. -1.]]
[ 0. -8.  0.  3.  0.  7.]
----------
2
3 2
[[ 0.00000000e+00 -1.77635684e-15  3.00000000e+00  0.00000000e+00
   1.00000000e+00 -7.00000000e+00]
 [ 1.00000000e+00  1.00000000e+00  2.00000000e+00  0.00000000e+00
   0.00000000e+00 -4.00000000e+00]
 [ 0.00000000e+00 -2.00000000e+00  1.00000000e+00  1.00000000e

In [870]:
#TODO побороть зацикливание

task5 = Task(
    [-1, 4, -3, 10], TaskType.MIN,
    [
        Inequality([1, 1, -1, -10], Signs.EQUAL, 0),
        Inequality([1, 14, 10, -10], Signs.EQUAL, 11),
    ]
)

result_5_value, result_5_point = solve(
    task5,
    [0, 0, 1, 1],
    [0, 0, 1, 1]
)

check(result_5_value, -4.0)
check(result_5_point, [1, 0, 1, 0])


[[  1.   1.  -1. -10.  -0.]
 [  1.  14.  10. -10. -11.]]
[  1.  -4.   3. -10.   0.]
(2, 5)
after gauss
[[ 0.          1.18181818  1.          0.         -1.        ]
 [-0.1        -0.21818182 -0.          1.          0.1       ]]
[ 0.         -9.72727273  0.          0.          4.        ]
----------
complete
functional:  -4.0
in point: [0, 0, 1.0, -0.1]
points are different


In [871]:
task_6 = Task(
    [-1, 5, 1, -1], TaskType.MIN,
    [
        Inequality([1, 3, 3, 1], Signs.LESS, 3),
        Inequality([0, 2, 3, -1], Signs.LESS, 4),
    ]
)

result_6_value, result_6_point = solve(
    task_6
)

check(result_6_value, -3)
check(result_6_point, [3, 0, 0, 0])


[[ 1.  3.  3.  1.  1.  0.  1.  0. -3.]
 [ 0.  2.  3. -1.  0.  1.  0.  1. -4.]]
[0, 0, 0, 0, 0, 0, -1, -1, 0]
(2, 9)
after gauss
[[ 1.  3.  3.  1.  1.  0.  1.  0. -3.]
 [ 0.  2.  3. -1.  0.  1.  0.  1. -4.]]
[ 1.  5.  6.  0.  1.  1.  0.  0. -7.]
----------
0
0 0
[[ 1.  3.  3.  1.  1.  0.  1.  0. -3.]
 [ 0.  2.  3. -1.  0.  1.  0.  1. -4.]]
[ 0.  2.  3. -1.  0.  1. -1.  0. -4.]
----------
1
1 0
[[ 0.33333333  1.          1.          0.33333333  0.33333333  0.
   0.33333333  0.         -1.        ]
 [-0.66666667  0.          1.         -1.66666667 -0.66666667  1.
  -0.66666667  1.         -2.        ]]
[-0.66666667  0.          1.         -1.66666667 -0.66666667  1.
 -1.66666667  0.         -2.        ]
----------
2
2 0
[[ 0.33333333  1.          1.          0.33333333  0.33333333  0.
   0.33333333  0.         -1.        ]
 [-1.         -1.          0.         -2.         -1.          1.
  -1.          1.         -1.        ]]
[-1. -1.  0. -2. -1.  1. -2.  0. -1.]
----------
3
5 1
[[ 0.33

In [872]:
task_7 = Task(
    [-1, -1, 1, -1, 2], TaskType.MIN,
    [
        Inequality([3, 1, 1, 1, -2], Signs.EQUAL, 10),
        Inequality([6, 1, 2, 3, -4], Signs.EQUAL, 20),
        Inequality([10, 1, 3, 6, -7], Signs.EQUAL, 30)
    ]
)

result_7_value, result_7_point = solve(
    task_7
)

check(result_7_value, -10)
check(result_7_point, [0, 0, 10, 0, 0])


[[  3.   1.   1.   1.  -2.   1.   0.   0. -10.]
 [  6.   1.   2.   3.  -4.   0.   1.   0. -20.]
 [ 10.   1.   3.   6.  -7.   0.   0.   1. -30.]]
[0, 0, 0, 0, 0, -1, -1, -1, 0]
(3, 9)
after gauss
[[  3.   1.   1.   1.  -2.   1.   0.   0. -10.]
 [  6.   1.   2.   3.  -4.   0.   1.   0. -20.]
 [ 10.   1.   3.   6.  -7.   0.   0.   1. -30.]]
[ 19.   3.   6.  10. -13.   0.   0.   0. -60.]
----------
0
0 2
[[ 0.   0.7  0.1 -0.8  0.1  1.   0.  -0.3 -1. ]
 [ 0.   0.4  0.2 -0.6  0.2  0.   1.  -0.6 -2. ]
 [ 1.   0.1  0.3  0.6 -0.7  0.   0.   0.1 -3. ]]
[ 0.   1.1  0.3 -1.4  0.3  0.   0.  -1.9 -3. ]
----------
1
1 0
[[ 0.          1.          0.14285714 -1.14285714  0.14285714  1.42857143
   0.         -0.42857143 -1.42857143]
 [ 0.          0.          0.14285714 -0.14285714  0.14285714 -0.57142857
   1.         -0.42857143 -1.42857143]
 [ 1.          0.          0.28571429  0.71428571 -0.71428571 -0.14285714
   0.          0.14285714 -2.85714286]]
[ 0.          0.          0.14285714 -0.1428571

In [873]:
# import sympy as sy
# a = np.array([sy.Rational(1), sy.Rational(2), sy.Rational(3)], dtype=sy.Rational)
# b = a / 17 / 19 * 17
# print(b)
# print(sy.Rational('6/5') + 1)